# Đồ án 2: Image Processing
**Họ tên: Nguyễn Tấn Lộc**\
**MSSV: 21127099**



Thư viện sử dụng:
- Numpy
- PIL
- Matplotlib
- Time

In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import time

# 1. Thay đổi độ sáng cho ảnh

In [2]:
def adjust_brightness(image_path):
    # Load ảnh
    image = Image.open(image_path)

    # Chuyển ảnh sang dạng array
    image_array = np.array(image)

    # Điều chỉnh độ tương phản
    adjusted_image_array = (image_array + np.mean(image_array))

    # Cắt các giá trị pixel chỉ được nằm trong phạm vi hợp lệ, ở đây là từ 0 tới 255
    adjusted_image_array = np.clip(adjusted_image_array, 0, 255)

    # Chuyển ảnh dạng array thành dạng ảnh 
    adjusted_image = Image.fromarray(adjusted_image_array.astype(np.uint8))

    return adjusted_image


# 2. Thay đổi độ tương phản

In [3]:
def change_contrast(image_path, contrast_factor):
    # Load ảnh
    image = Image.open(image_path)

    # Chuyển ảnh sang dạng array
    image_array = np.array(image)

    # Điều chỉnh độ sáng
    # Cách 1:
    # adjusted_image_array = image_array * float(contrast_factor)

    # Cách 2:
    #adjusted_image_array = (image_array * float(contrast_factor))-np.mean(image_array)
    
    # Cách 3:
    adjusted_image_array = (image_array - np.mean(image_array)) * float(contrast_factor) + np.mean(image_array)

    # Cắt các giá trị pixel chỉ được nằm trong phạm vi hợp lệ, ở đây là từ 0 tới 255
    adjusted_image_array = np.clip(adjusted_image_array, 0, 255)

    # Chuyển ảnh dạng array thành dạng ảnh
    adjusted_image = Image.fromarray(adjusted_image_array.astype(np.uint8))

    return adjusted_image


# 3. Lật ảnh (ngang - dọc)

In [4]:
def flip_image(image_path, direction):
    # Load ảnh
    image = Image.open(image_path)

    # Chuyển ảnh sang dạng array
    image_array = np.array(image)

    # Lật ảnh theo hướng được chỉ định
    # horizontal - lật ngang
    if direction == 'h':
        flipped_image = np.flip(image_array, axis=1)
        #flipped_image = image_array[:,::-1,:]

    # vertical - lật dọc
    elif direction == 'v':
        flipped_image = np.flip(image_array, axis=0)
        #flipped_image = image_array[::-1,:,:]
        
    else:
        raise ValueError("Không hợp lệ.")

    # Chuyển ảnh dạng array thành dạng ảnh 
    flipped_image = Image.fromarray(flipped_image)

    return flipped_image

# 4. Chuyển đổi ảnh RGB thành ảnh xám

Nguồn: https://www.tutorialspoint.com/dip/grayscale_to_rgb_conversion.htm

In [5]:
def convert_to_gray(image_path):
    # Load ảnh
    image = Image.open(image_path)

    # Chuyển ảnh sang dạng array
    image_array = np.array(image)

    # Áp dụng các trọng số
    gray_array = np.dot(image_array, [0.3, 0.59, 0.11])
    
    gray_array = np.minimum(gray_array, 255)
    # Chuyển đổi mảng thành ảnh PIL
    gray_image = Image.fromarray(gray_array.astype(np.uint8))

    return gray_image

# 5. Chuyển đổi ảnh RGB thành ảnh sepia

Nguồn: https://stackoverflow.com/questions/23802725/using-numpy-to-apply-a-sepia-effect-to-a-3d-array

In [6]:
def convert_to_sepia(image_path):
    # Load ảnh
    image = Image.open(image_path)

    # Chuyển đổi ảnh thành dạng array
    image_array = np.array(image)

    # Tính toán giá trị màu sepia dựa trên công thức
    sepia_array = np.dot(image_array, np.array([[0.393, 0.769, 0.189],
                                                [0.349, 0.686, 0.168],
                                                [0.272, 0.534, 0.131]]).T)

    # Đảm bảo giá trị màu không vượt quá 255
    sepia_array = np.minimum(sepia_array, 255) # Cái này giống với np.clip (em đang muốn thử cách khác np.clip)

    # Chuyển đổi mảng thành ảnh PIL
    sepia_image = Image.fromarray(sepia_array.astype(np.uint8))

    return sepia_image

# 6. Làm mờ nét ảnh

Nguồn: https://en.wikipedia.org/wiki/Kernel_(image_processing)

In [7]:
def convolve(channel, kernel):
    # Sử dụng np.pad để thêm phần đệm xung quanh kênh
    padded_channel = np.pad(channel, 1, mode='edge')

    # Áp dụng chập tại đây
    result = np.zeros_like(channel, dtype=float)
    for i in range(3):
        for j in range(3):
            result += padded_channel[i:i+channel.shape[0], j:j+channel.shape[1]] * kernel[i, j]

    return result

In [8]:
def blurred_image(image_path):
    # Load ảnh
    image = Image.open(image_path)

    # Chuyển đổi ảnh thành dạng array
    image_array = np.array(image)

    # Gaussian blur
    blurred_image_array = gaussian_blur(image_array)

    # Chuyển đổi mảng thành ảnh PIL
    blurred_image = Image.fromarray(blurred_image_array)

    return blurred_image

def gaussian_blur(img):
    # Định nghĩa blur kernel
    kernel = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]], dtype=float)
    #kernel = np.array([[1, 4,  6,  4,  1], [4, 16, 24, 16, 4], [4, 26, 36, 26, 4], [4, 16, 24, 16, 4], [1, 4,  6,  4,  1]], dtype=float)
    kernel /= (np.sum(kernel))  # Đây là 1/16

    # Áp dụng kernel cho từng kênh riêng
    blurred_image = np.zeros_like(img, dtype=float)

    for channel in range(img.shape[2]):
        blurred_image[:, :, channel] = convolve(img[:, :, channel], kernel)

    # Normalize lại
    blurred_image = np.clip(blurred_image, 0, 255)
    blurred_image = blurred_image.astype(np.uint8)
    
    return blurred_image

# 7. Làm sắc nét ảnh

In [9]:
def sharpen_image(image_path):
    # Load ảnh
    image = Image.open(image_path)

    # Chuyển đổi ảnh thành dạng array
    image_array = np.array(image)

    # Sharpen
    sharpened_image_array = sharpen(image_array)

    # Chuyển đổi mảng thành ảnh PIL
    sharpened_image = Image.fromarray(sharpened_image_array)

    return sharpened_image

def sharpen(img):
    # Định nghĩa sharpen kernel
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=float)

    # Áp dụng kernel cho từng kênh riêng
    sharpened_image = np.zeros_like(img, dtype=float)

    for channel in range(img.shape[2]):
        sharpened_image[:, :, channel] = convolve(img[:, :, channel], kernel)

    # Normalize lại
    sharpened_image = np.clip(sharpened_image, 0, 255)
    sharpened_image = sharpened_image.astype(np.uint8)

    return sharpened_image


# 8. Cắt ảnh theo kích thước (cắt ở trung tâm)

In [10]:
def crop_image_center(image_path, new_width, new_height):
    # Load ảnh
    image = Image.open(image_path)

    # Chuyển đổi ảnh thành dạng array
    image_array = np.array(image)

    # Lấy kích thước của ảnh gốc
    original_height, original_width, _ = image_array.shape

    # Tính toán điểm bắt đầu để cắt ảnh
    start_x = (original_width - new_width) // 2
    start_y = (original_height - new_height) // 2

    # Tính toán điểm kết thúc để cắt ảnh
    end_x = start_x + new_width
    end_y = start_y + new_height

    # Truy xuất phần ảnh cần cắt
    cropped_image_array = image_array[start_y:end_y, start_x:end_x, :]

    # Chuyển đổi mảng thành ảnh PIL
    cropped_image = Image.fromarray(cropped_image_array)

    return cropped_image


# 9. Cắt ảnh theo khung tròn

Cách 1: Thực ra cách 1 này không nên dùng vì hình tròn thì không cần xoay.

In [11]:
def rotate_point_circle(x, y, angle_deg, x_center, y_center):
    angle_rad = np.radians(angle_deg)
    x_rot = (x - x_center) * np.cos(angle_rad) - (y - y_center) * np.sin(angle_rad) + x_center
    y_rot = (x - x_center) * np.sin(angle_rad) + (y - y_center) * np.cos(angle_rad) + y_center
    return x_rot, y_rot

def crop_image_with_circle_1(image_path, x_center, y_center, radius, angle):
    # Load ảnh
    image = Image.open(image_path)

    # Chuyển đổi ảnh thành dạng array
    image_array = np.array(image)

    # Kích thước của ảnh
    height, width, _ = image_array.shape

    # Tạo mask
    mask = np.zeros((height, width), dtype=np.uint8)

    # Tính tọa độ xoay cho từng điểm và cập nhật mask
    for y in range(height):
        for x in range(width):
            x_rot, y_rot = rotate_point_circle(x, y, angle, x_center, y_center)

            if (x_rot - x_center)**2 + (y_rot - y_center)**2 <= radius**2:
                mask[y, x] = 255

    # Dùng mask để đối chiếu 
    cropped_image_array = np.where(mask[:, :, np.newaxis], image_array, 0)

    # Chuyển đổi mảng thành ảnh PIL
    cropped_image = Image.fromarray(cropped_image_array)

    return cropped_image

Cách 2

In [12]:
def crop_image_with_circle_2(image_path, radius):
    # Load ảnh
    image = Image.open(image_path)

    # Chuyển đổi ảnh thành dạng array
    image_array = np.array(image)

    # Kích thước của ảnh
    height, width = image_array.shape[:2]

    # Tạo lưới 2D từ chiều rộng và chiều cao của ảnh
    y, x = np.ogrid[:height, :width]

    # Tính khoảng cách từ mỗi điểm trên lưới đến tâm của hình tròn (ở giữa ảnh)
    distance = np.sqrt((x - width/2)**2 + (y - height/2)**2)

    # Tạo mask, chỉ giữ lại những điểm có khoảng cách đến tâm nhỏ hơn hoặc bằng bán kính
    mask = distance <= radius

    # Đặt các pixel không nằm trong mask thành màu đen (0, 0, 0)
    image_array[~mask] = [0, 0, 0]

    # Chuyển đổi mảng thành ảnh PIL
    cropped_image = Image.fromarray(image_array)


    return cropped_image

# 10. Cắt ảnh theo khung là 2 hình elip chéo nhau

Em dựa trên cách 1 của <u>Cắt ảnh theo khung tròn</u> để thực hiện cắt ảnh theo khung là 2 hình elip chéo nhau

In [13]:
def rotate_point_ellipes(x, y, angle_deg):
    angle_rad = np.radians(angle_deg)
    x_rot = x * np.cos(angle_rad) - y * np.sin(angle_rad)
    y_rot = x * np.sin(angle_rad) + y * np.cos(angle_rad)
    return x_rot, y_rot

def crop_image_with_ellipses(image_path, a1, b1, angle1, a2, b2, angle2):
    # Load ảnh
    image = Image.open(image_path)

    # Chuyển đổi ảnh thành dạng array
    image_array = np.array(image)

    height, width, _ = image_array.shape
    
    # Tạo mask
    mask = np.zeros((height, width), dtype=np.uint8)

    # Tính tọa độ xoay cho từng điểm và cập nhật mask
    for y in range(height):
        for x in range(width):
            x1_rot, y1_rot = rotate_point_ellipes(x - width / 2, y - height / 2, angle1)
            x2_rot, y2_rot = rotate_point_ellipes(x - width / 2, y - height / 2, angle2)

            if (x1_rot / a1) ** 2 + (y1_rot / b1) ** 2 <= 1 or (x2_rot / a2) ** 2 + (y2_rot / b2) ** 2 <= 1:
                mask[y, x] = 255

    # Dùng mask để đối chiếu 
    cropped_image_np = np.where(mask[:, :, np.newaxis], image_array, 0)

    # Chuyển đổi mảng thành ảnh PIL
    cropped_image = Image.fromarray(cropped_image_np)

    return cropped_image

# Hàm xuất hình ảnh

In [14]:
def generate_image(image_path,result):
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    axs[0].imshow(Image.open(image_path))
    axs[0].set_title("Ảnh gốc")
    axs[0].axis("off")

    axs[1].imshow(result, cmap="gray")
    axs[1].set_title("Ảnh đã chỉnh sửa")
    axs[1].axis("off")

    plt.tight_layout()
    plt.show()

# Hàm main

In [15]:
def main():
    # Người dùng nhập tên file ảnh để xử lý
    image_path = input("Nhập tên tập tin ảnh: ")

    # Người dùng nhập chức năng xử lý ảnh
    print("Chọn chức năng xử lý ảnh:")
    print("1. Thay đổi độ sáng")
    print("2. Thay đổi độ tương phản")
    print("3. Lật ảnh (ngang hoặc dọc)")
    print("4. Chuyển đổi ảnh sang ảnh xám hoặc sepia")
    print("5. Áp dụng làm mờ hoặc làm sắc nét")
    print("6. Cắt ảnh theo kích thước (cắt ở trung tâm)")
    print("7. Cắt ảnh theo khung hình tròn")
    print("8. Cắt ảnh theo khung hình ellip chéo nhau")
    print("0. Tất cả chức năng")
    print("------------------------------------------------------------------")
    function_choice = int(input("Chọn chức năng: "))
    
    # Thực hiện chức năng đã chọn
    if function_choice == 1:
        # Thay đổi độ sáng
        start= time.time()
        processed_image = adjust_brightness(image_path)
        end= time.time()
        print("1. Thay đổi độ sáng")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_brightness.png')

    elif function_choice == 2:
        # Thay đổi độ tương phản
        # Nhập số dương để tăng tương phản
        # Nhập số âm để giảm tương phản)
        # contrast_value = float(input("Nhập giá trị độ tương phản: "))
        contrast_value=2
        start= time.time()
        processed_image = change_contrast(image_path, contrast_value)
        end=time.time()
        print("2. Thay đổi độ tương phản")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_contrast.png')

    elif function_choice == 3:
        # Lật ảnh
        flip_direction = input("Nhập hướng lật (h cho lật ngang, v cho lật dọc): ")
        #flip_direction = 'v'
        start=time.time()
        processed_image = flip_image(image_path, flip_direction)
        end=time.time()
        print("3. Lật ảnh:")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_flip.png')

    elif function_choice == 4:
        # Chuyển đổi ảnh sang ảnh xám hoặc sepia
        print("Chọn chế độ chuyển đổi:")
        print("1. Ảnh xám")
        print("2. Ảnh sepia")
        color_choice = int(input("Nhập số chế độ chuyển đổi: "))
        if color_choice == 1:
            start=time.time()
            processed_image = convert_to_gray(image_path)
            end=time.time()
            print("4.1. Chuyển đổi ảnh sang ảnh xám")
            print(f"Thời gian chạy: {end-start}")
            result=generate_image(image_path, processed_image)
            processed_image.save(image_path[:image_path.rfind('.')] + '_gray.png')
        elif color_choice == 2:
            start=time.time()
            processed_image = convert_to_sepia(image_path)
            end=time.time()
            print("4.2. Chuyển đổi ảnh sang ảnh sepia")
            print(f"Thời gian chạy: {end-start}")
            result=generate_image(image_path, processed_image)
            processed_image.save(image_path[:image_path.rfind('.')] + '_sepia.png')
        else:
            print("Không hợp lệ")

    elif function_choice == 5:
        # Áp dụng làm mờ hoặc làm sắc nét
        print("Chọn chế độ xử lý ảnh:")
        print("1. Làm mờ")
        print("2. Làm sắc nét")
        process_choice = int(input("Nhập số chế độ xử lý ảnh: "))
        if process_choice == 1:
            start=time.time()
            processed_image = blurred_image(image_path)
            end=time.time()
            print("5.1. Áp dụng làm mờ")
            print(f"Thời gian chạy: {end-start}")
            result=generate_image(image_path, processed_image)
            processed_image.save(image_path[:image_path.rfind('.')] + '_blur.png')
        elif process_choice == 2:
            start=time.time()
            processed_image = sharpen_image(image_path)
            end=time.time()
            print("5.2. Áp dụng làm sắc nét")
            print(f"Thời gian chạy: {end-start}")
            result=generate_image(image_path, processed_image)
            processed_image.save(image_path[:image_path.rfind('.')] + '_sharp.png')
        else:
            print("Không hợp lệ")

    elif function_choice == 6:
        # Cắt ảnh theo kích thước (cắt ở trung tâm)
        # Thường là cắt theo hình vuông thì chiều rộng và chiều cao sẽ bằng nhau
        # new_width = int(input("Nhập chiều rộng: "))
        # new_height = int(input("Nhập chiều cao: "))
        new_width = 250
        new_height = 250
        start=time.time()
        processed_image = crop_image_center(image_path, new_width, new_height)
        end=time.time()
        print("6. Cắt ảnh theo kích thước (cắt ở trung tâm)")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_crop_center.png')

    elif function_choice == 7:
        # Cắt ảnh theo khung hình tròn

        # Cách 1
        # x_center, y_center, radius, angle = 256, 256, 256, 45
        # processed_image = crop_image_with_circle_1(image_path, x_center, y_center, radius, angle)

        #Cách 2
        radius=256
        start=time.time()
        processed_image = crop_image_with_circle_2(image_path, radius)
        end=time.time()
        print("7. Cắt ảnh theo khung hình tròn")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_crop_circle_method_2.png')

    elif function_choice == 8:
        # Cắt ảnh theo khung hình elip chéo nhau
        a1, b1, angle1 = 315, 180, 45
        a2, b2, angle2 = 315, 180, 135
        start=time.time()
        processed_image = crop_image_with_ellipses(image_path, a1, b1, angle1, a2, b2, angle2)
        end=time.time()
        print("8. Cắt ảnh theo khung hình ellip chéo nhau")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_crop_ellipes.png')

    elif function_choice == 0:
        # Thực hiện tất cả chức năng
        # Ở chức năng này, các giá trị thông số sẽ là em tự đặt.
        contrast_value = 1.5
        flip_direction_1 = "h"
        flip_direction_2 = "v"
        new_width = new_height = 300# chiều rộng và chiều cao sẽ bằng nhau
        x_center, y_center, radius, angle = 256, 256, 256, 45
        a1, b1, angle1 = 315, 180, 45
        a2, b2, angle2 = 315, 180, 135

        # Thay đổi độ sáng
        start=time.time()
        processed_image = adjust_brightness(image_path)
        end=time.time()
        print("1. Thay đổi độ sáng")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_brightness.png')

        # Thay đổi độ tương phản
        start=time.time()
        processed_image = change_contrast(image_path, contrast_value)
        end=time.time()
        print("2. Thay đổi độ tương phản")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_contrast.png')

        # Lật ảnh ngang - dọc
        start=time.time()
        processed_image = flip_image(image_path, flip_direction_1)
        end=time.time()
        print("3.1. Lật ảnh ngang")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_flip_horizontal.png')

        start=time.time()
        processed_image = flip_image(image_path, flip_direction_2)
        end=time.time()
        print("3.2. Lật ảnh dọc")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_flip_vertical.png')

        # Chuyển đổi ảnh thành ảnh xám hoặc ảnh sephia
        start=time.time()
        processed_image = convert_to_gray(image_path)
        end=time.time()
        print("4.1. Chuyển đổi ảnh thành ảnh xám")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_gray.png')

        start=time.time()
        processed_image = convert_to_sepia(image_path)
        end=time.time()
        print("4.2. Chuyển đổi ảnh thành ảnh sepia")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_sepia.png')
        
        # Làm mờ hoặc làm sắc nét ảnh
        start=time.time()
        processed_image = blurred_image(image_path)
        end=time.time()
        print("5.1. Làm mờ ảnh")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_blur.png')

        start=time.time()
        processed_image = sharpen_image(image_path)
        end=time.time()
        print("5.1. Làm sắc nét ảnh")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_sharp.png')

        # Cắt ảnh ở trung tâm
        start=time.time()
        processed_image = crop_image_center(image_path, new_width, new_height)
        end=time.time()
        print("6. Cắt ảnh ở trung tâm")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_crop_center.png')

        # Cắt ảnh theo khung tròn
        start=time.time()
        processed_image = crop_image_with_circle_1(image_path, x_center, y_center, radius, angle)
        end=time.time()
        print("6. Cắt ảnh theo khung tròn")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_crop_circle_method_1.png')
        
        # Cắt ảnh theo khung 2 elip chéo nhau
        start=time.time()
        processed_image = crop_image_with_ellipses(image_path, a1, b1, angle1, a2, b2, angle2)
        end=time.time()
        print("7. Cắt ảnh theo khung 2 elip chéo nhau")
        print(f"Thời gian chạy: {end-start}")
        result=generate_image(image_path, processed_image)
        processed_image.save(image_path[:image_path.rfind('.')] + '_crop_ellipes.png')
    else:
        print("Không hợp lệ")


In [16]:
if __name__ == "__main__":
    main()

Chọn chức năng xử lý ảnh:
1. Thay đổi độ sáng
2. Thay đổi độ tương phản
3. Lật ảnh (ngang hoặc dọc)
4. Chuyển đổi ảnh sang ảnh xám hoặc sepia
5. Áp dụng làm mờ hoặc làm sắc nét
6. Cắt ảnh theo kích thước (cắt ở trung tâm)
7. Cắt ảnh theo khung hình tròn
8. Cắt ảnh theo khung hình ellip chéo nhau
0. Tất cả chức năng
------------------------------------------------------------------


ValueError: invalid literal for int() with base 10: ''